# Data Adquisition

## Setup

### Library import


In [ ]:
import os
import zipfile

import geopandas as gpd
import lonboard as lb
import requests
from lonboard.colormap import apply_categorical_cmap, apply_continuous_cmap
from palettable.colorbrewer.sequential import Blues_8

## Utils

In [ ]:
def download_and_extract_zip(url, directory):
    """
    Download a zip file from a URL and extract it to a directory.
    """
    # Make directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)

    # Extract zip file name from url
    zip_file_name = os.path.basename(url)

    # Download the file
    response = requests.get(url)
    zip_file_path = os.path.join(directory, zip_file_name)
    with open(zip_file_path, "wb") as f:
        f.write(response.content)

    # Unzip the file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(directory)

    # Remove the zip file
    os.remove(zip_file_path)

# Climate Resilience 
## South Sudan
### Get data
#### [GADM data (version 4.1)](https://gadm.org/download_country.html)

**Read data**

In [ ]:
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_SSD_0.json"
ssd = gpd.read_file(url)

**Save data**

In [ ]:
ssd.to_file("../data/processed/South Sudan/gadm41_SSD_0.geojson", driver="GeoJSON")

#### [HydroRIVERS v1.0](https://www.hydrosheds.org/products/hydrorivers/)

**Download data**

In [ ]:
url = "https://data.hydrosheds.org/file/HydroRIVERS/HydroRIVERS_v10_af_shp.zip"
directory = "../data/raw/Climate Resilience/South Sudan/Additional Data"

download_and_extract_zip(url, directory)

**Prepare data**

In [ ]:
file = "../data/raw/Climate Resilience/South Sudan/Additional Data/HydroRIVERS_v10_af_shp/\
    HydroRIVERS_v10_af.shp"
ssd_bbox = tuple(ssd.total_bounds)

# Read the shapefile and filter by the bounding box of South Sudan
gdf = gpd.read_file(file, bbox=ssd_bbox, engine="fiona")
# Intersect with the South Sudan shapefile
gdf = gpd.overlay(gdf, ssd, how="intersection")

gdf.head()

**Save data**

In [ ]:
gdf.to_file(
    "../data/processed/South Sudan/HydroRIVERS_v10_af_SSD.geojson", driver="GeoJSON"
)

In [ ]:
gdf = gpd.read_file("../data/processed/South Sudan/HydroRIVERS_v10_af_SSD.geojson")

**Display on map**

In [ ]:
layer = lb.PathLayer.from_geopandas(gdf)
layer.get_color = apply_continuous_cmap(gdf["ORD_STRA"] / 8, Blues_8)
layer.get_width = gdf["ORD_STRA"]
layer.width_scale = 3000
layer.width_min_pixels = 0.5

m = lb.Map(layers=[layer])
m

#### [Aqueduct Global Maps 3.0 Data](https://www.wri.org/data/aqueduct-global-maps-30-data)

**Download data**

In [ ]:
url = "https://wri-projects.s3.amazonaws.com/Aqueduct30/finalData/Y2019M07D12_Aqueduct30_V01.zip"
directory = "../data/raw/Climate Resilience/South Sudan/Additional Data"

download_and_extract_zip(url, directory)

**Prepare data**

In [ ]:
file = "../data/processed/South Sudan/HydroRIVERS_v10_af_SSD.geojson"
gdf = gpd.read_file(
    file,
)

**Read data**

In [ ]:
file = "../data/processed/South Sudan/riverine_flood_risk_aqueduct03.geojson"
ssd_bbox = tuple(ssd.total_bounds)

# Read the shapefile and filter by the bounding box of South Sudan
gdf = gpd.read_file(file, bbox=ssd_bbox, engine="fiona")
# Intersect with the South Sudan shapefile
gdf = gpd.overlay(gdf, ssd, how="intersection")

gdf.head()

**Save data**

In [ ]:
gdf.to_file(
    "../data/processed/South Sudan/riverine_flood_risk_aqueduct03_SSD.geojson",
    driver="GeoJSON",
)

In [ ]:
gdf = gpd.read_file(
    "../data/processed/South Sudan/riverine_flood_risk_aqueduct03_SSD.geojson"
)

**Display on map**

In [ ]:
gdf["rfr_cat"].unique()

In [ ]:
color_map = {
    0.0: [255, 255, 153],
    1.0: [255, 230, 0],
    2.0: [255, 153, 0],
    3.0: [255, 25, 0],
    4.0: [204, 0, 20],
}

In [ ]:
layer = lb.SolidPolygonLayer.from_geopandas(gdf)

layer.get_fill_color = apply_categorical_cmap(
    values=gdf["rfr_cat"].astype("category"), cmap=color_map, alpha=150
)

m = lb.Map(layer)
m